# This notebook is a utility to help with the CFA data processing to digitise core logs

In [1]:
import os

import numpy as np
import scipy as scp
import glob

import pandas as pd
import xarray as xr


import time
import datetime
import panel as pn
pn.extension('ipywidgets','plotly','tabulator',template='fast')

In [2]:
#file for core break db
cores_db_file=pn.widgets.FileInput(accept=".json", filename="test_data/20221216/cuttinglog_db.json") #json to avoid a weird csv file could just have 

cores_db=pd.DataFrame(columns=["Date", "Core-ID","Bag Number", "Length Before (mm)", "Length After", "Top Depth", "Notes","Breaks"])


content_fun = lambda row:pn.widgets.Tabulator(pd.DataFrame(row['Breaks']),sortable=False)
cores_db_table=pn.widgets.Tabulator(cores_db,sortable=False,hidden_columns=['Breaks'],row_content=content_fun,embed_content=True,page_size=5,pagination='local')
        
cd = pn.widgets.DatePicker(
    name= "Cutting Date:",
    value=datetime.date.today() 
)    

cbn=pn.widgets.IntInput(
    name="Bag Number:",
    value=1
)

cid=pn.widgets.TextInput(
    name="Core-ID:",
    value="DSS0506"
)

lb = pn.widgets.IntInput(
    value=1000,
    start=0,
    end=1200,
    step=1, 
 name='Length Before:'
 )

la=pn.widgets.IntInput(
    value=1000,
    start=0,
    end=1200,
    step=1, 
    name='Length After:'
 )

breaks_editors = {
    'position': {'type':'number','max':20,'step':1},
    'length removed': {'type':'number','max':20,'step':1}
}

breaks_tmp=pd.DataFrame({'position':[np.NaN,0,0,0,0,0,np.NaN],
                         'length removed':[0,0,0,0,0,0,0]},index=['Top','1','2','3','4','5','Bottom'])
breaks=pn.widgets.Tabulator(breaks_tmp,
    show_index=True,
    editors=breaks_editors
    
)


lr = pn.widgets.IntInput(
 value=0,
 start=0,
 end=1000,
 name='Amount removed (mm):'
 )

top_depth = pn.widgets.FloatInput(
 value=0,
 start=0,
 step=0.01,
 name='Top Depth (m):'
 )


notes_txt=pn.widgets.TextAreaInput(
    name='Notes',
    max_rows=10,
    rows=5,
    value="Notes.."
)

save_db_btn=pn.widgets.Button(name='Save',button_type="primary")
add_entry_btn=pn.widgets.Button(name='Add new entry',button_type="primary")


def add_new_log_entry(event,cores_db):
    print('button clicked')
    newdata={"Date": cd.value, "Core-ID":cid.value,"Bag number": cbn.value, "Length Before": lb.value, "Length After": la.value,'Top Depth': top_depth.value, 'Notes':notes_txt.value,'Breaks':breaks.value}
    newentry=pd.DataFrame(newdata)
    cores_db=pd.concat([cores_db,newentry])
 
        
add_entry_btn.on_click([add_new_log_entry,cores_db])      



Watcher(inst=Button(button_type='primary', name='Add new entry'), cls=<class 'panel.widgets.button.Button'>, fn=[<function add_new_log_entry at 0x7b7c0849a700>, Empty DataFrame
Columns: [Date, Core-ID, Bag Number, Length Before (mm), Length After, Top Depth, Notes, Breaks]
Index: []], mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [3]:
coresbox=pn.WidgetBox('Exisiting stuff',cores_db_table)
tbox= pn.WidgetBox('Input core breaks',pn.Row(pn.Column(pn.Row(cd,cid),pn.Row(top_depth, cbn),pn.Row(notes_txt,pn.Column(lb,la)),add_entry_btn),breaks))
display(pn.Column(cores_db_file,save_db_btn,coresbox,tbox))



Column
    [0] FileInput(accept='.json', filename='test_data/20221216/cuttin...)
    [1] Button(button_type='primary', name='Save')
    [2] WidgetBox
        [0] Markdown(str)
        [1] Tabulator(embed_content=True, hidden_columns=['Breaks'], page_size=5, pagination='local', row_content=<function <lambda> a..., sortable=False, value=Empty DataFrame
Columns: [...)
    [3] WidgetBox
        [0] Markdown(str)
        [1] Row
            [0] Column
                [0] Row
                    [0] DatePicker(name='Cutting Date:', value=datetime.date(2024, ...)
                    [1] TextInput(name='Core-ID:', value='DSS0506')
                [1] Row
                    [0] FloatInput(name='Top Depth (m):', start=0, step=0.01)
                    [1] IntInput(name='Bag Number:', value=1)
                [2] Row
                    [0] TextAreaInput(max_rows=10, name='Notes', rows=5, value='Notes..')
                    [1] Column
                        [0] IntInput(end=1200, name='Length Before:', start=0, value=1000)
                        [1] IntInput(end=1200, name='Length After:', start=0, value=1000)
                [3] Button(button_type='primary', name='Add new entry')
            [1] Tabulator(editors={'position': {'type': 'num...}, value=        position  ...)

In [4]:
from panel.viewable import Viewer

class BreakInput(Viewer):

    cores_db_dir="test_data/DSS0506_64U"
    cores_db_file_list=glob.glob(cores_db_dir+"/*.json")
    cores_db_file=pn.widgets.Select(name="Cutting log file",options=cores_db_file_list,value=cores_db_file_list[0])
    
   # cores_db_file=pn.widgets.FileInput(accept=".json", filename="test_data/20221216/cuttinglog_db.json") #json to avoid a weird csv file could just have some other text
    

    cores_db=pd.DataFrame(columns=["Date","Core-ID","Bag Number","Length Before","Length After",'Top Depth','Notes','Breaks'])
     
    cores_table_formatters = {
            'Date': {'type': 'date'}
    }
 
    
    breaks_editors = {
        'Position': {'type':'number','max':20,'step':1},
        'Length Removed': {'type':'number','max':20,'step':1}
    }

    breaks_tmp=pd.DataFrame({'Position':[None,0,0,0,0,0,None],
                         'Length Removed':[0,0,0,0,0,0,0],'Break':['Top','1','2','3','4','5','Bottom']}).set_index('Break')
    
    breaks=pn.widgets.Tabulator(breaks_tmp,
        show_index=False,
        editors=breaks_editors
    )
 
    cd = pn.widgets.DatePicker(
        name= "Cutting Date:",
        value=datetime.date.today() 
    )    

    cid=pn.widgets.TextInput(
        name="Core ID:",
        value="",
    )   

    cbn=pn.widgets.IntInput(
        name="Bag Number:",
        value=1
    )

    lb = pn.widgets.IntInput(
        value=1000,
        start=0,
        end=1200,
        step=1, 
        name='Length Before (mm):'
    )

    la=pn.widgets.IntInput(
        value=1000,
        start=0,
        end=1200,
        step=1, 
        name='Length After (mm):'
     )

    top_depth = pn.widgets.FloatInput(
        value=0,
        start=0,
        step=0.01,
        name='Top Depth (m):'
    )

    notes_txt=pn.widgets.TextAreaInput(
        name='Notes',
        max_rows=10,
        rows=5,
        value="Notes.."
    )    

    
    def __init__(self,**params):
        super().__init__(**params)
       # self.cores_db=self.get_core_db()
        #self.cbn.value=len(self.cores_db.index)+1# set bag num automatically to the next
        #if self.cbn.value>1:
        #    self.cid.value=self.cores_db['Core-ID'].iloc[0]
       #self.content_fun = lambda row:pn.widgets.Tabulator(pd.DataFrame(row['Breaks']).set_index('Break'),sortable=False)
        self.content_fun = lambda row:pn.widgets.Tabulator(pd.DataFrame(row['Breaks']),sortable=False,show_index=False)
        self.cores_db_table=pn.widgets.Tabulator(self.cores_db,sortable=False,hidden_columns=['Breaks'],row_content=self.content_fun,embed_content=True,page_size=5,pagination='local',
                                                 show_index=False,formatters=self.cores_table_formatters)
        self.save_db_btn.on_click(self.save_core_db)
        self.add_entry_btn.on_click(self.add_new_log_entry)
        self.get_db_btn.on_click(self.get_core_db)

    
    def add_new_log_entry(self,event):
        breaks_df=pd.DataFrame.from_dict(self.breaks.value)
        breaks_dict=self.breaks_to_dict(self.breaks.value)
        new_core=pd.DataFrame({"Date": [self.cd.value], "Core-ID":[self.cid.value],"Bag Number": [self.cbn.value], "Length Before": [self.lb.value], "Length After": [self.la.value],'Top Depth': [self.top_depth.value], 'Notes':[self.notes_txt.value],'Breaks':[breaks_dict]})
        if not self.check_length_removed(self.lb.value,self.la.value,self.breaks.value):
            print('total length error')
            return
        if self.cbn.value in self.cores_db['Bag Number'].values:
            print('error')           
        else: 
            self.cores_db=pd.concat([self.cores_db,new_core],ignore_index=True)
        self.cores_db_table.value=self.cores_db
        self.cbn.value=self.cbn.value+1
    
    def get_core_db(self,event):
        #return self.cores_db
        if os.path.exists(self.cores_db_file.value):
            self.cores_db=pd.read_json(self.cores_db_file.value)
            self.cores_db_table.value=self.cores_db
            self.cbn.value=len(self.cores_db.index)+1# set bag num automatically to the next
            self.cid.value=self.cores_db['Core-ID'].iloc[0]
           # self.cores_db=pd.read_json(self.cores_db_file.filename)
            

    def save_core_db(self,event):
        self.cores_db.to_json(self.cores_db_file.value)


    def check_length_removed(self,lb,la,breaks):
        total_removed=breaks['Length Removed'].sum()
        if lb-(la+total_removed)>3:
            return False
        else:
            return True
    
    def breaks_to_dict(self,breaks_df):
        tmp={}
        tmp["Break"]=breaks_df.index.to_flat_index().values.tolist()
        for col in breaks_df.columns:
            tmp[col]=breaks_df[col].values.tolist()
        return tmp 
    
    save_db_btn=pn.widgets.Button(name="Save Logs", button_type="primary")  
    get_db_btn=pn.widgets.Button(name="Open Log database", button_type="primary")  
    add_entry_btn=pn.widgets.Button(name='Add new entry',button_type="primary")
    
    def __panel__(self):
        #selector to load db from file and button to save db
        #content_fun = lambda row:pn.widgets.Tabulator(pd.DataFrame(row['Breaks']).set_index('Break'),sortable=False)
        #cores_db_table=pn.widgets.Tabulator(self.cores_db,sortable=False,hidden_columns=['Breaks'],row_content=content_fun,embed_content=True,page_size=5,pagination='local')
        coresbox=pn.WidgetBox('Exisiting stuff',self.cores_db_table)
        tbox= pn.WidgetBox('Input core breaks',pn.Row(pn.Column(pn.Row(self.cd,self.cid),pn.Row(self.top_depth, self.cbn),pn.Row(self.notes_txt,pn.Column(self.lb,self.la)),self.add_entry_btn),self.breaks))
        return pn.Column(pn.Row(self.cores_db_file,self.get_db_btn,self.save_db_btn,),coresbox,tbox)
        
    

In [5]:
BreakInput(name="Input Cutting logs").servable()

Column
    [0] Row
        [0] Select(name='Cutting log file', options=['test_data/DSS0506_64U/cu...], value='test_data/DSS0506_64U/cut...)
        [1] Button(button_type='primary', name='Open Log database')
        [2] Button(button_type='primary', name='Save Logs')
    [1] WidgetBox
        [0] Markdown(str)
        [1] Tabulator(embed_content=True, formatters={'Date': {'type': '...}, hidden_columns=['Breaks'], page_size=5, pagination='local', row_content=<function BreakInput.__ini..., show_index=False, sortable=False, value=Empty DataFrame
Columns: [...)
    [2] WidgetBox
        [0] Markdown(str)
        [1] Row
            [0] Column
                [0] Row
                    [0] DatePicker(name='Cutting Date:', value=datetime.date(2024, ...)
                    [1] TextInput(name='Core ID:')
                [1] Row
                    [0] FloatInput(name='Top Depth (m):', start=0, step=0.01)
                    [1] IntInput(name='Bag Number:', value=1)
                [2] Row
                    [0] TextAreaInput(max_rows=10, name='Notes', rows=5, value='Notes..')
                    [1] Column
                        [0] IntInput(end=1200, name='Length Before (mm):', start=0, value=1000)
                        [1] IntInput(end=1200, name='Length After (mm):', start=0, value=1000)
                [3] Button(button_type='primary', name='Add new entry')
            [1] Tabulator(editors={'Position': {'type': 'num...}, show_index=False, value=        Position  ...)

In [28]:
testcores=pd.read_json("test_data/20221216/cuttinglog_db.json")

In [29]:
print(testcores.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           2 non-null      datetime64[ns]
 1   Core-ID        2 non-null      object        
 2   Bag number     2 non-null      int64         
 3   Length Before  2 non-null      int64         
 4   Length After   2 non-null      int64         
 5   Top Depth      2 non-null      int64         
 6   Notes          2 non-null      object        
 7   Breaks         2 non-null      object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 260.0+ bytes
None
